<a href="https://colab.research.google.com/github/mirianfsilva/machine-learning/blob/master/final-project/hmm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biological Sequence Analysis using Profile Hidden Markov Models
### Machine Learning Final Project
##### Mírian Francielle da Silva

In this project I used **Hidden Markov Model** (HMM), a probabilistic model to analyze **multiple sequence alignment** (msa) of proteins.

In the model, each column of symbols in the alignment is represented by a frequency distribution of the symbols (called a "state"), and insertions and deletions are represented by other states. One moves through the model along a particular path from state to state in a Markov chain (i.e., random choice of next move), trying to match a given sequence. The next matching symbol is chosen from each state, recording its probability (frequency) and also the probability of going to that state from a previous one (the transition probability). 

State and transition probabilities are multiplied to obtain a probability of the given sequence. The hidden nature of the HMM is due to the lack of information about the value of a specific state, which is instead represented by a probability distribution over all possible values.

When designing a HMM, the most important criteria are the number of states and the transitions between them.

### HMM Problems:
To analyse and describe data with HMMs, three major problems must be solved. 
- _What is the probability of an observed sequence according to a given model?_ 
This can be calculated using the forward algorithm and the result can be used, among others, to decide which model in a collection of HMMs best describes an observation.
- _The next problem is to find the most probable state sequence to have emitted an observation._
This state sequence can be used to determine the process which has caused the observation and is calculated with the Viterbi algorithm. 
- _The third problem is to optimize the model, to maximize the probabilities of given observations._
The best-known method for HMM training is the Baum-Welch, aka, Expectation Maximization training algorithm.


In [1]:
#Hidden Markov Models learn library
%pip install hmmlearn

     |████████████████████████████████| 153kB 43.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/2c/b6/0e/63a865a30e21e01d04f417d8995fbfb793d6bd464707efc546
Successfully built hmmlearn


In [2]:
#Python tools for computational molecular biology
%pip install biopython

     |████████████████████████████████| 2.2MB 47.7MB/s 


In [3]:
#Another Hidden Markov Model library
%pip install pomegranate

     |████████████████████████████████| 5.2MB 42.7MB/s 


In [0]:
#For better visualizations
#%pip install pygraphviz

In [0]:
# Important modules 
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range, input

import argparse, sys

import matplotlib
import matplotlib as mpl
import matplotlib.style
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd 
import numpy as np
import Bio
import timeit

from Bio.Seq import Seq
from Bio import SeqIO
from sklearn.utils import shuffle
from scipy.io import loadmat
from scipy.stats import norm
from pomegranate import *

from hmmlearn import hmm
from multiprocessing import Pool, TimeoutError

In [5]:
print("Biopython v" + Bio.__version__)

Biopython v1.73


#### About dataset: 

The [Pfam](http://pfam.xfam.org/) database is a large collection of protein families, each represented by multiple sequence alignments and hidden Markov models (HMMs). 

[Pfam References](http://pfam.xfam.org/help?tab=helpReferencesBlock):
[The Pfam protein families database in 2019](https://academic.oup.com/nar/article/47/D1/D427/5144153): S. El-Gebali, J. Mistry, A. Bateman, S.R. Eddy, A. Luciani, S.C. Potter, M. Qureshi, L.J. Richardson, G.A. Salazar, A. Smart, E.L.L. Sonnhammer, L. Hirsh, L. Paladin, D. Piovesan, S.C.E. Tosatto, R.D. Finn
Nucleic Acids Research (2019)  doi: 10.1093/nar/gky995

__Dataset used: [Family: Miga (PF10265)](http://pfam.xfam.org/family/PF10265#alignBlock)__
Mitoguardin (Miga) was first identified in flies as a mitochondrial outer-membrane protein that promotes mitochondrial fusion. 

#### About file format: 
In bioinformatics and biochemistry, the FASTA format is a text-based format for representing either nucleotide sequences or amino acid (protein) sequences, in which nucleotides or amino acids are represented using single-letter codes. The format also allows for sequence names and comments to precede the sequences.

In [0]:
input_file = "PF10265_seed.fasta"
input_file_high_lenght = "miga.fasta"
input_file_small_lenght = "zincFinger.fasta"

#### Parsing sequence file formats:

In [10]:
def fasta_reader(input_file):
    from Bio.SeqIO.FastaIO import FastaIterator
    with open(input_file_small_lenght) as handle:
        for record in FastaIterator(handle):
            yield record
msa = []
for entry in fasta_reader(input_file_small_lenght):
    msa.append(np.array(list(entry.seq)))
    #msa_train.append(entry.seq)
    print ("ID:", str(entry.id), "\t Lenght:", str("{:,d}".format(len(entry))) ) #This is header of fasta entry
    #print (repr(entry.seq), "\n") #This is sequence of specific fasta entry
    print (str(entry.seq), "\n") #This is sequence of specific fasta entry

ID: ACE2_YEAST/603-627 	 Lenght: 31
FECLY-PNCN---KVFKRRYNIRSHIQT--H 

ID: ACE2_YEAST/633-657 	 Lenght: 31
YSCDF-PGCT---KAFVRNHDLIRHKIS--H 

ID: ADR1_YEAST/104-126 	 Lenght: 31
FVCE---VCT---RAFARQEHLKRHYRS--H 

ID: ADR1_YEAST/132-155 	 Lenght: 31
YPCG---LCN---RCFTRRDLLIRHAQK-IH 

ID: B5RIE4_DROME/222-244 	 Lenght: 31
FTCK---ICS---RSFGYKHVLQNHERT--H 

ID: B5RIE4_DROME/306-328 	 Lenght: 31
YTCE---ICD---GKFSDSNQLKSHMLV--H 

ID: BNC1_HUMAN/720-743 	 Lenght: 31
FQCD---ICK---KTFKNACSVKIHHKN-MH 

ID: BNC1_HUMAN/928-951 	 Lenght: 31
ITCH---LCQ---KTYSNKGTFRAHYKT-VH 

ID: BRLA_EMENI/350-375 	 Lenght: 31
HVCWV-PGCH---RAFSRSDNLNAHYTK-TH 



A profile HMM is used to align a sequence to a reference 'profile', where the reference profile can either be a single sequence, or an alignment of many sequences (genomes). In essence, this profile has a 'match' state for every position in the reference profile, and 'insert' state, and a 'delete' state. The insert state allows the external sequence to have an insertion into the sequence without throwing off the entire alignment, such as the following.